# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

clang: error: argument to '-isysroot' is missing (expected 1 value)
clang: error: no input files
ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -isysroot;/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX14.4.sdk   -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
clang: error: argument to '-isysroot' is missing (expected 1 value)
clang: error: no input files
With exit code 0


In [2]:
# %load python/drawingTools.py


In [14]:
do_fixed_rate_eff = False
do_turnon = False

In [26]:
# %load samples.py
import ROOT
import pandas as pd

samples = [
    HistoFile('doubleele_flat1to100_PU200_hgc', 'PU200', 'v152C.100H', 'ele-gun'),
]

In [27]:
for smp in samples:
    smp.print_primitives()


<HistoFile ../plots/histos_doubleele_flat1to100_PU200_hgc_v152C.100H.root, ele-gun>
--- HistoClass: 
['Cluster3DHistos', 'ResoHistos', 'HistoSetEff']
--- TPs: 
['HGCCl3d']
--- TP selections:
{   'HGCCl3d': [   'IDEmLoosePt30',
                   'IDEmLooseIDPuVetoPt30',
                   'IDEmPfPt20',
                   'IDEmPfIDPuVetoPt30',
                   'IDEmPfPt30',
                   'IDEmPfIDPuVetoPt20',
                   'all',
                   'IDEmTightIDPuVetoPt30',
                   'IDEmPfIDPuVeto',
                   'IDEmLoose',
                   'IDEmLooseIDPuVeto',
                   'IDEmLoosePt20',
                   'IDPuVetoPt20',
                   'Pt30',
                   'Pt20',
                   'IDPuVetoPt30',
                   'IDEmTight',
                   'IDEmTightPt20',
                   'IDEmPf',
                   'IDEmLooseIDPuVetoPt20',
                   'IDEmTightIDPuVeto',
                   'IDPuVeto',
                   'IDEmTightI

In [28]:
import python.collections as collections
import python.selections as selections
# this is needed to get all the labels defined in cfg modules
# import python.plotters_config 
from cfg import hgctps

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]





In [29]:
import python.calibrations as calib

if do_fixed_rate_eff:
    calib_mgr = calib.CalibManager()
    calib_mgr.set_pt_wps_version('data/rate_pt_wps_v151C.81A.json')
    rate_pt_wps = calib_mgr.get_pt_wps()
    print(rate_pt_wps)

    for obj in rate_pt_wps.keys():
        data_selections = calib.rate_pt_wps_selections(
                    rate_pt_wps, obj, 'pt_em')
    selm = selections.SelectionManager()
    labels_dict.update(selm.get_labels())


In [30]:
%%time

from python import histos

hplot = HPlot(samples, labels_dict)
hplot.create_histo_proxies(histos.HistoSetEff)



--- Cluster3DHistos
# of plots: 528
# of primitives: 264
--- ResoHistos
# of plots: 1584
# of primitives: 264
--- GenParticleHistos
# of plots: 1584
# of primitives: 264
CPU times: user 66.2 ms, sys: 5.26 ms, total: 71.5 ms
Wall time: 69.8 ms


In [31]:
%%time
if do_turnon:
    for smp in hplot.data['sample'].unique():
        print ('Sample: {}'.format(smp))
        for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
            for tp in tps:
    #         for tp in ['EG', 'TkEleEL']:
                print ('PU: {}, TP: {}'.format(pu, tp))

                for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():

                    if 'Pt' not in tp_sel:
                        continue
    #                 tp_sel_den = 'all'
                    # print tp_sel
                    tp_sel_den = tp_sel.split('Pt')[0]
                    if tp_sel_den == '':
                        tp_sel_den = 'all'
                    print ('  {} den -> {}'.format(tp_sel, tp_sel_den) )
    #                 for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD', 'GENEtaC', 'GENEtaB']:
                        if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                            continue
    #                     print gen_sel
                        hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                        hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                        hset[0][0].computeTurnOn(hsetden[0][0].h_num)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 2.62 µs


In [32]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele-gun,PU200,HGCCl3d,IDEmPfPt30,GENEtaBCD,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4eb00>
1,ele-gun,PU200,HGCCl3d,IDEmTightPt30,GENEtaC,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4eef0>
2,ele-gun,PU200,HGCCl3d,IDEmLoosePt20,GENEtaBC,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4f3d0>
3,ele-gun,PU200,HGCCl3d,Pt20,GENEtaABC,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4ef20>
4,ele-gun,PU200,HGCCl3d,IDEmPfIDPuVetoPt30,GENEtaD,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4d6f0>
5,ele-gun,PU200,HGCCl3d,IDEmLooseIDPuVeto,GENEtaBCD,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4f9d0>
6,ele-gun,PU200,HGCCl3d,IDEmPfIDPuVetoPt20,GENEtaA,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4ecb0>
7,ele-gun,PU200,HGCCl3d,Pt20,GEN,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4f4c0>
8,ele-gun,PU200,HGCCl3d,IDPuVetoPt20,GENPt30,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4ebc0>
9,ele-gun,PU200,HGCCl3d,IDEmTightPt20,GENEtaBC,<class 'python.histos.HistoSetEff'>,<drawingTools.HProxy object at 0x32fe4d480>


In [33]:
hplot.data.tp.unique()

array(['HGCCl3d'], dtype=object)

In [34]:
hplot.data.tp_sel.unique()

array(['IDEmPfPt30', 'IDEmTightPt30', 'IDEmLoosePt20', 'Pt20',
       'IDEmPfIDPuVetoPt30', 'IDEmLooseIDPuVeto', 'IDEmPfIDPuVetoPt20',
       'IDPuVetoPt20', 'IDEmTightPt20', 'IDEmLooseIDPuVetoPt20',
       'IDEmTightIDPuVeto', 'IDEmLoosePt30', 'IDEmLoose', 'IDEmTight',
       'IDPuVeto', 'IDPuVetoPt30', 'IDEmLooseIDPuVetoPt30', 'Pt30', 'all',
       'IDEmPfPt20', 'IDEmTightIDPuVetoPt20', 'IDEmPf', 'IDEmPfIDPuVeto',
       'IDEmTightIDPuVetoPt30'], dtype=object)

In [35]:
hplot.data.gen_sel.unique()

array(['GENEtaBCD', 'GENEtaC', 'GENEtaBC', 'GENEtaABC', 'GENEtaD',
       'GENEtaA', 'GEN', 'GENPt30', 'GENEtaB', 'GENEtaABCD', 'GENPt15'],
      dtype=object)

In [36]:
hplot.data['sample'].unique()

array(['ele-gun'], dtype=object)

In [48]:
import os
wc = wp.WebPageCreator(
    topic_dir='eff', 
    project_dir='fp131Xv3-CMSSW14.0.X-ARValid', 
#     project_dir='test', 
    base_dir='/Users/cerminar/CERNbox/www/plots/', 
    tmp_dir=os.environ['TMPDIR'])

In [44]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.15)
newconfig.marker_styles.append(10)

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-gun'], 'PU200', 
    ['HGCCl3d'], 
    ['all', 'IDPuVeto','IDEmPfIDPuVeto', 'IDEmLooseIDPuVeto'], ['GENEtaBC'], debug=False)


# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.draw(text=text, 
        x_min=0, x_max=100,
        y_min=0., y_max=1.1,
#         v_lines=[1.52, 1.7],
        h_lines=[0.9, 1])

# dm.write(name='hEffVsPt_HGCCl3d_CTL1ID')
# dm.toWeb(name='hEffVsPt_HGCCl3d_CTL1ID', page_creator=wc)



In [47]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.15)
newconfig.marker_styles.append(10)

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-gun'], 'PU200', 
    ['HGCCl3d'], 
    ['all', 'IDPuVeto','IDEmPfIDPuVeto', 'IDEmLooseIDPuVeto'], ['GENPt30'], debug=False)


# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

dm.draw(text=text, 
        # x_min=0, x_max=100,
        y_min=0., y_max=1.1,
#         v_lines=[1.52, 1.7],
        h_lines=[0.9, 1])

# dm.write(name='eg_TDRvsSummer20_matchig_eff')



In [ ]:
wc.publish()

## Effinciency vs $\eta$

In [25]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HGCCl3d'], ['all'], ['GENEe'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [14]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEe'], debug=False)

labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-match'], 'PU200', 
    ['HGCCl3d', 'TkCl3DMatch'], 
    ['PUId'], ['GENEe'], debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

print(text)




dm.draw(text=text, 
        x_min=1.4, x_max=2.5,
        y_min=0., y_max=1.1,
#         v_lines=[1.52, 1.7],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')



PU200, PUId


In [15]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEePt15'], debug=False)

labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)



hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-match'], 'PU200', 
    ['HGCCl3d', 'TkCl3DMatch'], 
    ['PUId'], ['GENEePt15'], debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

print(text)


dm.draw(text=text, 
        x_min=1.4, x_max=2.5,
        y_min=0., y_max=1.1,
#         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')



PU200, PUId, p_{T}^{GEN}>=15GeV


In [16]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)



hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEePt30'], debug=False)

labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-match'], 'PU200', 
    ['HGCCl3d', 'TkCl3DMatch'], 
    ['PUId'], ['GENEePt30'], debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

print(text)



dm.draw(text=text, 
        x_min=1.4, x_max=2.5,
        y_min=0., y_max=1.1,
        v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')



PU200, PUId, p_{T}^{GEN}>=30GeV


In [17]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)



hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEePt15'], debug=False)

labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)



hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-match'], 'PU200', 
    ['HGCCl3d', 'TkCl3DMatch'], 
    ['PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE995MTkPt5'], ['GENEePt15'], debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

print(text)

dm.draw(text=text, 
        x_min=1.4, x_max=2.5,
        y_min=0.7, y_max=1.1,
#         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')



PU200, p_{T}^{GEN}>=15GeV


In [18]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)





hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-match', 'ele'], 'PU200', 
    ['TkCl3DMatch', 'HGCCl3d' ], 
    ['PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE995MTkPt5', 'EgBdtLE995PUId'], ['GENEePt30'], debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['ele'], 'PU200', 
#     ['HGCCl3d' ], 
#     ['PUId', 'EgBdtLE995PUId'], ['GENEePt30'], debug=False)

# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEePt30'], debug=False)

labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


print(text)

dm.draw(text=text, 
        x_min=1.4, x_max=2.5,
        y_min=0.7, y_max=1.1,
#         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')



PU200, TkEle EE, Tight-TkEle, p_{T}^{GEN}>=30GeV


In [47]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.3, 0.2)
newconfig.legend_position=(0.2, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)


# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['ele', 'ele-match'], 'PU200', 
#     ['HGCCl3d'], 
#     ['all'], ['GENEeEtaBC'], debug=False)



hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-match', 'ele', ], 'PU200', 
    ['TkCl3DMatch', 'HGCCl3d'], 
    ['all', 'PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE995MTkPt5', 'EgBdtLE995PUId'], ['GENEeEtaBC'], debug=False)

for hset in hsets:
    hset.computeEff(rebin=2)
    
new_labels = []  
for lb in labels:
    lb.split(',')[1:]
    print((',').join(lb.split(',')[1:]))
    new_labels.append((',').join(lb.split(',')[1:]))
    
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=new_labels)

print(text)




# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['tkele'], 'PU200', 
#     ['TkEleEE'], 
#     ['IDTightE'], ['GENEeEtaBC'], debug=False)

# labels = ['TkEle']

# for hset in hsets:
#     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)


dm.draw(text=text, 
        x_min=0., x_max=50,
        y_min=0., y_max=1.1,
#         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')
# dm.draw(text=text, 
#         x_min=0, x_max=100,
#         y_min=0., y_max=1.1,
# #         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
#         h_lines=[0.9, 1])




 TkCl3DMatch, PUId
 TkCl3DMatch, PUId, BDT^{eg}_{LE}@995%
 TkCl3DMatch, PUId, BDT^{eg}_{LE}@995%, (p_{T}^{Tk} > 2GeV & #Chi^{2}_{norm} < 15) | p_{T}^{Tk} > 5GeV 
 TkCl3DMatch, PUId, BDT^{eg}_{LE}@995%, p_{T}^{Tk} > 5GeV
 HGC Cl3d
 HGC Cl3d, PUId
 HGC Cl3d, BDT^{eg}_{LE}@995%, PUId
PU200, 1.52 < |#eta^{GEN}| #leq 2.4


In [12]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.32, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)




hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele',], 'PU200', 
    ['HGCCl3d'], 
    ['EgBdtLE975PUId', 'EgBdtLE995PUId'], ['GENEeEtaBC'], debug=False)

labels=['STA (tight WP)' ,'STA (loose WP)']

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

print(len(hsets))

print(text)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-match' ], 'PU200', 
    ['TkCl3DMatch'], 
    ['PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE995MTkPt5'], ['GENEeEtaBC'], debug=False)

labels = ['3Dcl-tk #DeltaR match w.'+lb for lb in labels]
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# print(text)




hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['tkele'], 'PU200', 
    ['TkEleEE'], 
    ['IDTightE'], ['GENEeEtaBC'], debug=False)

labels = ['Tk-ele (elliptic match)']
text = "PU200, 1.52 < |#eta^{GEN}| <= 2.4"
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)


dm.draw(text=text, 
#         x_min=1.4, x_max=2.5,
        y_min=0., y_max=1.1,
#         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
        h_lines=[0.9, 1])
#         options='hist')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')
# dm.draw(text=text, 
#         x_min=0, x_max=100,
#         y_min=0., y_max=1.1,
# #         v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
#         h_lines=[0.9, 1])




No match found for: sample: ['ele'] pu: PU200, tp: ['HGCCl3d'], tp_sel: ['EgBdtLE975PUId', 'EgBdtLE995PUId'], gen_sel: ['GENEeEtaBC'], classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [23]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele', 'ele-match'], ['PU200'], ['HGCCl3d', 'TkCl3DMatch'], ['PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE975MTkPt2Chi2Pt5', 'PUIdEgBdtLE995Pt1MTkPt5', 'PUIdEgBdtLE995Pt2MTkPt5'], ['GENEePt30'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [29]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele', 'ele-match'], ['PU200'], ['HGCCl3d', 'TkCl3DMatch'], ['PUId', 'PUIdEgBdtLE995', 'PUIdEgBdtLE995MTkPt2Chi2Pt5', 'PUIdEgBdtLE975MTkPt2Chi2Pt5', 'PUIdEgBdtLE995Pt1MTkPt5', 'PUIdEgBdtLE995Pt2MTkPt5'], ['GENEeEtaBC'])
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0., max_y=1.1, h_lines=[0.9, 1])

In [11]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HGCCl3d'], ['all'], ['GENEePt30'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0, max_y=1.1, 
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [19]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HGCCl3d'], ['all'], ['GENEePt15'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0, max_y=1.1, 
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [50]:
'EgBdtHE900',
                   'EgBdtHE975',
                   'EgBdtLE950',
                   'EgBdtHE950',
                   'Em',
                   'EgBdtLE995',
                   'all',
                   'EgBdtLE975',
                   'EgBdtLE900',
                   'EgBdtHE995']}

IndentationError: unexpected indent (1489013771.py, line 2)

In [22]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Em', 'EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    ['GENEePt15'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0, max_y=1.1,
     min_x=1.3, max_x=2.7,
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [24]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Em', 'EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    ['GENEePt30'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0, max_y=1.1,
     min_x=1.3, max_x=2.7,
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [25]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Em', 'EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    ['GENEePt15'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0, max_y=1.1,
     min_x=1.3, max_x=2.7,
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [27]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Em', 'EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    ['GENEePt30'])
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], 
     labels, 
     text=text, 
     min_y=0.8, max_y=1.1,
     min_x=1.3, max_x=2.7,
     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

## Efficiency vs $p_{T}$

In [55]:
rleg_config.marker_styles.extend([24, 25, 26, 27, 28, 29])
rleg_config.colors.extend([1,2,3,4,5,6,7])


AttributeError: 'ColorPalette' object has no attribute 'extend'

In [14]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Eg','EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [15]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['all', 'Em','EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


No match found for: sample: pho pu: ['PU200'], tp: ['HGCCl3d'], tp_sel: ['all', 'Em', 'EgBdtLE975', 'EgBdtLE995', 'EgBdtLE975PUId', 'EgBdtLE995PUId'], gen_sel: GENEeEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [58]:
  'EgBdtLE975EtaBC@100kHz',
                   'EgBdtLE995EtaBC@1000kHz',
                   'EgBdtLE900',
                   'EgBdtLE975EtaBC@20kHz',
                   'EgBdtLE995EtaBC@100kHz',
                   'EgBdtHE995',
                   'EgBdtLE975EtaBC@1000kHz']}
                

IndentationError: unexpected indent (2919696389.py, line 2)

In [31]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975EtaBC@20kHz', 'EgBdtLE995EtaBC@20kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [32]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@20kHz', 'EgBdtLE995PUIdEtaBC@20kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [35]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@20kHz', 'EgBdtLE995PUIdEtaBC@20kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [60]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975EtaBC@100kHz', 'EgBdtLE995EtaBC@100kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [38]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@100kHz', 'EgBdtLE995PUIdEtaBC@100kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [39]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@100kHz', 'EgBdtLE995PUIdEtaBC@100kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [33]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975EtaBC@1000kHz', 'EgBdtLE995EtaBC@1000kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [37]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'ele', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@1000kHz', 'EgBdtLE995PUIdEtaBC@1000kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [34]:
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    'pho', 
    ['PU200'], 
    ['HGCCl3d'], 
    ['EgBdtLE975PUIdEtaBC@1000kHz', 'EgBdtLE995PUIdEtaBC@1000kHz'], 
    'GENEeEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, 
     min_y=0, max_y=1.1, 
     y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


## Turn-on curves

In [62]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HMvDRshapeDrCalib'], ['EmPt20'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: None pu: ['PU200'], tp: ['HMvDRshapeDrCalib'], tp_sel: ['EmPt20'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [63]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V10', ['PU200'], ['HMvDRshapeDrCalib'], ['EmPt20'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: ele-V10 pu: ['PU200'], tp: ['HMvDRshapeDrCalib'], tp_sel: ['EmPt20'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [64]:
hplot.labels_dict.update({'HMvDRshapeDrCalibNew': 'new calib'})

In [65]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V11', ['PU200'], ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], ['EmPt20'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: ele-V11 pu: ['PU200'], tp: ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], tp_sel: ['EmPt20'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [66]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V11', ['PU200'], ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], ['EmPt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: ele-V11 pu: ['PU200'], tp: ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], tp_sel: ['EmPt30'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [67]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HMvDRshapeDrCalib'], ['EmPt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: None pu: ['PU200'], tp: ['HMvDRshapeDrCalib'], tp_sel: ['EmPt30'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [68]:
print rleg_config.colors
print colors
colors.extend([8,9,10,11, 12, 13, 14, 15])
# print rleg_config.marker_styles.extend([33, 33, 33, 33, 33, 33, 33])

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2505138390.py, line 1)

In [69]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['HMvDRshapeDrCalib'], ['EmPt10', 'EmPt20', 'EmPt30'], 'GENEtaBCD', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: None pu: ['PU200'], tp: ['HMvDRshapeDrCalib'], tp_sel: ['EmPt10', 'EmPt20', 'EmPt30'], gen_sel: GENEtaBCD, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

## Scaling

In [70]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(1, 0, 50)

tf_yc.SetParLimits(3, 0.9, 1)
tf_yc.SetParLimits(0, 0, 1)
tf_yc.SetParLimits(2, 0.1, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [71]:
fake_histoname = uuid.uuid4().hex[:6]


In [72]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


# tps_scaling = ['EG', 'TkEleEL']
# tp_sel_scaling = [('EGq3Pt{}', 'EGq3'), ('EGq4Pt{}', 'EGq4'), ('EGq5Pt{}', 'EGq5')]
# pt_points = [10, 15, 20, 25, 30, 40]

tps_scaling = ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew']
tp_sel_scaling = [('EmPt{}', 'Em')]
pt_points = [10, 20, 30]

# sample = 'calib'
# tps_scaling = ['HMvDRshapeDrCalib']
# tp_sel_scaling = [('EmPt{}', 'Em')]
# pt_points = [10, 20, 30]

for sample in samples:
    print sample.type
    for pu in ['PU0', 'PU200']:
        for gen_sel in ['GENEtaBC', 'GENEtaBCD', 'GENEtaB', 'GENEtaC']:
            for tp in tps_scaling:
                for tp_sel_strings in tp_sel_scaling:
                    pt_95 = []
                    pt_95_err = []
                    for pt in pt_points:
                        tp_sel = tp_sel_strings[0].format(pt)
                        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample.type, pu, tp, tp_sel, gen_sel)
                            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            #                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                        if hsets is None:
                            continue
            
    #                     if pt == 20:
                        tf_yc.SetParameters(0.6, pt, 0.1, 1.0, 0.0)
    #                     else:
    #                         tf_yc.SetParameters(1, 20, 4, 1.0, 0.0)
                    
                        h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

                        result = h_eff_vs_pt[0].Fit(tf_yc, 'EMQRS+')

                        grint = ROOT.TGraphErrors(h_eff_vs_pt[0].GetN())
                        stuff.append(grint)
                        grint.SetTitle("Fitted line with .95 conf. band")
                        for point in range(0, h_eff_vs_pt[0].GetN()):
                            grint.SetPoint(point, h_eff_vs_pt[0].GetX()[point], 0)
                        ROOT.TVirtualFitter.GetFitter().GetConfidenceIntervals(grint)


                        print '------- {}'.format( text)
                        print result.Print()
                        pt095 = tf_yc.GetX(0.95)
                        pt_95.append(pt095)

                        prev_err = 0
                        error = 0
                        for idx,point in enumerate(grint.GetX()):
    #                         print idx, point, grint.GetEY()[idx]
                            if pt095 > point:
                                prev_err = grint.GetEY()[idx]
                            else:
                                error = (prev_err+grint.GetEY()[idx])/2
                                break
                        pt_95_err.append(error)
                        print '       pt th: {}, 95% eff: {}, err: {}'.format(pt, pt095, error)
                        #                 points.append((tp, pu, pt, pt095))
                        draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                        ROOT.gStyle.SetOptFit(11111)
                        grint.Draw('same')

#                     print len(pt_points)
#                     print len(pt_95)
#                     print len(pt_95_err)
#                     print pt_95_err
                    if len(pt_95) == 0:
                        continue
                    graph = ROOT.TGraphErrors(len(pt_points), array.array('d', pt_points), 
                                                              array.array('d', pt_95),
                                                              array.array('d', [0.]*len(pt_points)),
                                                              array.array('d', pt_95_err) )
                    graph.SetMarkerStyle(7)
                    graph.SetMarkerColor(2)
                    stuff.append(graph)

                    hplot.data = hplot.data.append({'sample': sample.type,
                                                    'pu': pu,
                                                    'tp': tp,
                                                    'tp_sel': tp_sel_strings[1],
                                                    'gen_sel': gen_sel,
                                                    'classtype': fake_histoname,
                                                    'histo': HWrapper(graph),},
                                                    ignore_index=True)

                
                

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (1917141814.py, line 28)

In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique()

In [ ]:
scaling_params = pd.DataFrame(columns=['sample', 'tp', 'tp_sel', 'params'])
ROOT.gStyle.SetOptFit(11111)

for sample in hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique():
    print sample
    for eta_region in ['EtaBC', 'EtaBCD', 'EtaB', 'EtaC']:
        for pu in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample)].pu.unique():
            for tp in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample) & (hplot.data.pu == pu)].tp.unique():
                for tp_sel in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                    hsets, labels, text = hplot.get_histo(fake_histoname, sample, pu, tp, tp_sel, ['GEN{}'.format(eta_region)])
                    result = hsets[0].Fit('pol1', 'MES+', '', 5, 35)
                    result.GetParams()
                    scaling_params = scaling_params.append({'sample': sample,
                                                            'pu': pu,
                                                             'tp': tp,
                                                             'tp_sel': tp_sel+eta_region,
                                                             'params': (result.GetParams()[0], result.GetParams()[1])},
                                                              ignore_index=True)
                    draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

In [73]:
scaling_params

NameError: name 'scaling_params' is not defined

In [74]:
scaling_params.to_csv(r'scaling_params_{}.csv'.format(version_V9))

NameError: name 'scaling_params' is not defined

In [75]:

import numpy as np
from array import array

def f_cb(x, par):
    return ROOT.Math.crystalball_function(x[0], par[0], par[1], par[2], par[3])

tf_cb = ROOT.TF1('cb', f_cb, 0, 100, 4)
tf_cb.SetParameters(10., 4., 3, 20.)

def f_cbi(x, par):
    return ROOT.Math.crystalball_cdf(x[0], par[0], par[1], par[2], par[3])
#     tf_cb.SetParameters(par[0], par[1], par[2], par[3])
#     npts = 1000
#     x_ = []
#     w_ = []
   
#     return tf_cb.IntegralFast(npts, np.array(npts), np.array(npts), 0., x[0])
    #return tf_cb.Integral(0., x[0])
    
    
tf_cbi = ROOT.TF1('cbi', f_cbi, 0, 100, 4)
tf_cbi.SetParameters(10., 4., 3, 20.)
tf_cbi.SetLineColor(4)

c = newCanvas()
tf_cbi.Draw()
c.Draw()


    # ROOT::Math::crystalball_function (double x, double alpha, double n, double sigma, double x0=0)

# fitf_cb = ROOT.TF1('cbfunc', 'ROOT.TMath.crystalball_function (x, [0], [1], [2], [3])', 0, 100)
# 

# fitf_cb.SetParameters(1, 1, 1, 0)


In [76]:
# Double_t turnon_func(Double_t *x, Double_t *par)
# {
#   double halfpoint = par[0];
#   double slope = par[1];
#   double plateau = par[2];

#   //double offset = par[3];
#   //double plateau = 1.0;
#   double offset = 0;

#   double pt = TMath::Max(x[0],0.000001);

#    double arg = 0;
#    //cout << pt <<", "<< halfpoint <<", " << slope <<endl;
#    arg = (pt - halfpoint)/(TMath::Sqrt(pt)*slope);
#    double fitval = offset+0.5*plateau*(1+TMath::Erf(arg));
#    return fitval;
# }




def turnon_func1(x, par):

    return 0.5 * par[0] * (1+ROOT.TMath.Erf((x[0]-par[1])/(par[2]*ROOT.TMath.Sqrt(2))))




def turnon_func(x, par):
  
    halfpoint = par[0]
    slope = par[1]
    plateau = par[2]
    offset = 0
    pt = ROOT.TMath.Max(x[0],0.000001)
    arg = 0
    arg = (pt - halfpoint)/(ROOT.TMath.Sqrt(pt)*slope)
    fitval = offset+0.5*plateau*(1+ROOT.TMath.Erf(arg))
    return fitval

fitf_to = ROOT.TF1('myfunc', turnon_func1, 0, 100, 3)
#fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf_to.SetParameters(1,25,2)
fitf_to.SetLineColor(3)

def fermi(x, params):
    return params[3]*(params[0] - 1/(ROOT.TMath.Exp((x[0]-params[1])/(params[2]))+1))
  
fitf = ROOT.TF1('myfunc', fermi, 0, 100, 4)
fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf.SetParameters(1,25,2,1)



c = newCanvas()
fitf.Draw()
c.Draw()


In [77]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib', 'DEFMerged']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3198567673.py, line 16)

In [ ]:
points

In [78]:
# print points

points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if (point[0]=='DEF' and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if (point[0]=='DEF' and 'PU200' in point[1])]

points_merged_pu0 = [point for point in points if (point[0]=='DEFMerged' and 'PU0' in point[1])]
points_merged_pu200 = [point for point in points if (point[0]=='DEFMerged' and 'PU200' in point[1])]

# print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

def drawScaling(points, label, text):
    graph =  ROOT.TGraph(len(points), array('d', [point[2] for point in points]), array('d',[point[3] for point in points]))
    drawGraphsSame([graph], labels=[label], min_y=0, max_y=50, options='A*', text=text)
    graph.Fit('pol1', 'S')
    
    ROOT.gStyle.SetOptFit(11111)
    return graph

g1 = drawScaling(points=points_nocalib_pu0, label='PU0', text='PU0: no calib')
g3 = drawScaling(points=points_calib_pu0, label='PU0', text='PU0: calib')
g5 = drawScaling(points=points_merged_pu0, label='PU0', text='PU0: merged')

g2 = drawScaling(points=points_nocalib_pu200, label='PU200', text='PU200: no calib')
g4 = drawScaling(points=points_calib_pu200, label='PU200', text='PU200: calib')
g6 = drawScaling(points=points_merged_pu200, label='PU200', text='PU200: merged')


# graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
# drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
# drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)


# graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
# drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
# drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

NameError: name 'points' is not defined

In [79]:
points = []


for pu in ['PU0', 'PU200']:
    for tp in ['HMvDR', 'HMvDRfixed']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (4048128504.py, line 13)

In [80]:
from array import array
points_calib_pu0 = [point for point in points if ('fixed' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'fixed' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('fixed' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'fixed' not in point[0] and 'PU200' in point[1])]

print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
draw([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
draw([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
draw([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
draw([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3123131289.py, line 8)

In [81]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleAAA', 'eleAA', 'eleBB', 'eleCC', 'eleDD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



NameError: name 'samples_ele' is not defined

In [82]:
titles_df

NameError: name 'titles_df' is not defined

In [83]:
dir(histo_df.ele['DEF'][0])


NameError: name 'histo_df' is not defined

In [84]:
histo_df.loc['DEF', 'ele']

NameError: name 'histo_df' is not defined

In [85]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



NameError: name 'tp_sets' is not defined

In [86]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

NameError: name 'particles' is not defined

In [87]:
histo_den_df

NameError: name 'histo_den_df' is not defined

In [88]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

NameError: name 'particles' is not defined

In [89]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

NameError: name 'particles' is not defined

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [90]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



NameError: name 'samples_photon' is not defined

In [91]:
dir(histo_df.photon['DEF'][0])


NameError: name 'histo_df' is not defined

In [92]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)


NameError: name 'histo_df' is not defined